In [1]:
import pandas as pd
from pandas import read_excel
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from datetime import date, datetime, timedelta, timezone
import copy
from pandas import IndexSlice as idx
pd.set_option('display.max_columns', None)  # 当列太多时不换行
from numpy import exp, nan
import quantstats as qs


import warnings
warnings.filterwarnings('ignore') # 忽略警告
import pandas as pd
from pandas import IndexSlice as idx

import talib as ta
# 计算natr
def natr(df, n):
    high = np.array([float(x) for x in df['high']])
    low = np.array([float(x) for x in df['low']])
    close = np.array([float(x) for x in df['close']])
    df['natr'] = ta.NATR(high, low, close, timeperiod=n)
    return df['natr']

/Users/yiwei/Desktop/git/Monterey/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 导入所需的包
import pandas as pd
import mysql.connector
from datetime import datetime
from tqdm import tqdm
import numpy as np


In [26]:
columns = ['code', 'name', 'pre_close', 'open', 'high', 'low', 'close', 'limit_up_down',
          'close_ma_5', 'bias_5', 'pct_chg', 'adj_factor', 'vol', 'vol_5', 'amount', 'amount_5',
          'volatility', 'code_stk', 'name_stk', 'pre_close_stk', 'open_stk', 'high_stk', 'low_stk',
          'close_stk', 'pct_chg_stk', 'adj_factor_stk', 'vol_stk', 'amount_stk', 'pe_ttm', 'pb',
          'ps_ttm', 'dv_ratio', 'total_share', 'float_share', 'total_mv', 'circ_mv', 'debt_to_assets',
          'volatility_stk', 'is_call', 'conv_price', 'conv_value', 'conv_prem', 'dblow', 'issue_size',
          'remain_size', 'remain_cap', 'turnover', 'turnover_5', 'cap_mv_rate', 'list_date', 'list_days',
          'conv_start_date', 'left_conv_start_days', 'conv_end_date', 'left_years', 'ytm', 'pure_value',
          'bond_prem', 'option_value', 'theory_value', 'theory_bias', 'rating', 'yy_rating', 'orgform',
          'area', 'industry_1', 'industry_2', 'industry_3', 'maturity_put_price', 'maturity',
          'popularity_ranking', 'pct_chg_5', 'pct_chg_5_stk', 'alpha_pct_chg_5', 'theory_conv_prem',
          'mod_conv_prem', 'open_pct_chg', 'high_pct_chg', 'low_pct_chg']

def get_date_by_date_range(start_date, end_date):
    cnx = mysql.connector.connect(user='root', password='St7950819',
                                  host='sh-cynosdbmysql-grp-g4mg9e64.sql.tencentcdb.com', port='25739',
                                  database='cb_schema')
    query = ("SELECT code, name, pre_close, open, high, low, close, limit_up_down, "
             "close_ma_5, bias_5, pct_chg, adj_factor, vol, vol_5, amount, amount_5, "
             "volatility, code_stk, name_stk, pre_close_stk, open_stk, high_stk, low_stk, "
             "close_stk, pct_chg_stk, adj_factor_stk, vol_stk, amount_stk, pe_ttm, pb, "
             "ps_ttm, dv_ratio, total_share, float_share, total_mv, circ_mv, debt_to_assets, "
             "volatility_stk, is_call, conv_price, conv_value, conv_prem, dblow, issue_size, "
             "remain_size, remain_cap, turnover, turnover_5, cap_mv_rate, list_date, list_days, "
             "conv_start_date, left_conv_start_days, conv_end_date, left_years, ytm, pure_value, "
             "bond_prem, option_value, theory_value, theory_bias, rating, yy_rating, orgform, "
             "area, industry_1, industry_2, industry_3, maturity_put_price, maturity, "
             "popularity_ranking, pct_chg_5, pct_chg_5_stk, alpha_pct_chg_5, theory_conv_prem, "
             "mod_conv_prem, open_pct_chg, high_pct_chg, low_pct_chg, trade_date "
             "FROM cb_schema.cb_data "
             "WHERE trade_date BETWEEN %s AND %s")

    try:
        cursor = cnx.cursor()
        cursor.execute(query, (start_date, end_date))
        
        # 创建空列表存储所有数据
        all_data = []
        all_dates = []
        
        # 读取所有数据
        for row in cursor:
            # 分离数据和日期
            data = list(row[:-1])  # 所有列除了最后的日期
            date = row[-1]         # 最后一列是日期
            
            all_data.append(data)
            all_dates.append(date)
        
        # 创建 DataFrame
        df = pd.DataFrame(all_data, columns=columns)
        
        # 添加日期列
        df['trade_date'] = all_dates
        
        # 设置 MultiIndex
        df = df.set_index(['code', 'trade_date'])
        
        # 转换数据类型
        # 转换日期列
        date_columns = ['list_date', 'conv_start_date', 'conv_end_date']
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])
        
        # 转换数值列（排除已知的非数值列）
        non_numeric_cols = ['code', 'name', 'code_stk', 'name_stk', 'is_call', 'rating', 
                        'yy_rating', 'orgform', 'area', 'industry_1', 'industry_2', 'industry_3']
        numeric_cols = [col for col in df.columns if col not in non_numeric_cols + date_columns]
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='ignore')
            
        return df
    except Exception as e:
        print(f'错误: {str(e)}')
        raise
    finally:
        cursor.close()
        cnx.close()
    return result_dict



def upload_cb_data_to_db(start_date, end_date, batch_size=1000):
    """
    将 cb_data.pq 中的数据上传到数据库
    
    参数:
    start_date: str, 格式 'YYYYMMDD'
    end_date: str, 格式 'YYYYMMDD'
    batch_size: int, 每批上传的数据量
    """
    # MySQL 保留字列表
    reserved_words = ['limit', 'order', 'desc', 'group', 'key', 'range', 'values']
    
    # 数据库连接配置
    db_config = {
        'user': 'root',
        'password': 'St7950819',
        'host': 'sh-cynosdbmysql-grp-ed8g46uu.sql.tencentcdb.com',
        'port': '21582',
        'database': 'cb_bond'
    }
    # 连接数据库
    print("连接数据库...")
    cnx = mysql.connector.connect(user='root', password='St7950819',
                                  host= 'sh-cynosdbmysql-grp-g4mg9e64.sql.tencentcdb.com', port= '25739',
                                  database='cb_schema')
    
    try:
        # 读取 Parquet 文件
        print("读取数据文件...")
        df = pd.read_parquet('/Users/yiwei/Desktop/git/cb_data.pq')

        # 重命名 limit 列
        if 'limit' in df.columns:
            df = df.rename(columns={'limit': 'limit_up_down'})
        
        # 确保索引被重置为列
        df = df.reset_index()
        
        # 过滤日期范围
        df['trade_date'] = pd.to_datetime(df['trade_date'])
        mask = (df['trade_date'] >= pd.to_datetime(start_date)) & (df['trade_date'] <= pd.to_datetime(end_date))
        df = df[mask]
        
        if len(df) == 0:
            print("指定日期范围内没有数据")
            return
        
        # 处理数据类型
        float_columns = df.select_dtypes(include=['float64']).columns
        for col in float_columns:
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        
        # 将日期列转换为正确的格式
        date_columns = ['trade_date', 'list_date', 'conv_start_date', 'conv_end_date']
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')
        
        # 准备插入语句，为保留字添加反引号
        columns = df.columns.tolist()
        columns_quoted = [f"`{col}`" if col.lower() in reserved_words else col for col in columns]
        placeholders = ', '.join(['%s'] * len(columns))
        columns_str = ', '.join(columns_quoted)
        
        insert_query = f"""
        INSERT INTO cb_schema.cb_data ({columns_str})
        VALUES ({placeholders})
        ON DUPLICATE KEY UPDATE
        """
        # 添加 ON DUPLICATE KEY UPDATE 子句，同样为保留字添加反引号
        update_str = ', '.join([f"`{col}` = VALUES(`{col}`)" if col.lower() in reserved_words else f"{col} = VALUES({col})" 
                              for col in columns if col not in ['id']])
        insert_query += update_str
        
    
        cursor = cnx.cursor()
        
        # 分批上传数据
        total_rows = len(df)
        num_batches = (total_rows + batch_size - 1) // batch_size
        
        print(f"开始上传数据，总共 {total_rows} 行，分 {num_batches} 批处理...")
        print("SQL 语句预览:")
        print(insert_query)
        
        for i in tqdm(range(0, total_rows, batch_size)):
            batch_df = df.iloc[i:i+batch_size]
            
            # 准备批量数据
            # batch_data = [tuple(x) for x in batch_df.replace({np.nan: None}).values]
            batch_data = []
            for _, row in batch_df.iterrows():
                row_data = []
                for value in row:
                    if pd.isna(value) or (isinstance(value, float) and (np.isinf(value) or np.isnan(value))):
                        row_data.append(None)
                    else:
                        row_data.append(value)
                batch_data.append(tuple(row_data))
            
            try:
                cursor.executemany(insert_query, batch_data)
                cnx.commit()
            except mysql.connector.Error as err:
                print(f"批次 {i//batch_size + 1} 出错: {err}")
                print("出错的数据示例:")
                print(batch_df.head())
                cnx.rollback()
                continue
        
        print("数据上传完成！")
        
    except Exception as e:
        print(f"发生错误: {str(e)}")
        raise
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'cnx' in locals():
            cnx.close()
            print("数据库连接已关闭")


pd.set_option('display.max_columns', None)  # 当列太多时不换行
df = pd.read_parquet('/Users/yiwei/Desktop/git/cb_data.pq') # 导入转债数据




In [29]:
upload_cb_data_to_db('20250101', '20260101')

连接数据库...
读取数据文件...
开始上传数据，总共 32193 行，分 33 批处理...
SQL 语句预览:

        INSERT INTO cb_schema.cb_data (code, trade_date, name, pre_close, open, high, low, close, limit_up_down, close_ma_5, bias_5, pct_chg, adj_factor, vol, vol_5, amount, amount_5, volatility, code_stk, name_stk, pre_close_stk, open_stk, high_stk, low_stk, close_stk, pct_chg_stk, adj_factor_stk, vol_stk, amount_stk, pe_ttm, pb, ps_ttm, dv_ratio, total_share, float_share, total_mv, circ_mv, debt_to_assets, volatility_stk, is_call, conv_price, conv_value, conv_prem, dblow, issue_size, remain_size, remain_cap, turnover, turnover_5, cap_mv_rate, list_date, list_days, conv_start_date, left_conv_start_days, conv_end_date, left_years, ytm, pure_value, bond_prem, option_value, theory_value, theory_bias, rating, yy_rating, orgform, area, industry_1, industry_2, industry_3, maturity_put_price, maturity, popularity_ranking, pct_chg_5, pct_chg_5_stk, alpha_pct_chg_5, theory_conv_prem, mod_conv_prem, open_pct_chg, high_pct_chg, low_pct_

100%|██████████| 33/33 [02:57<00:00,  5.39s/it]

数据上传完成！
数据库连接已关闭


In [34]:
df= get_date_by_date_range('20220801', '20230102')

print("DataFrame 信息:")
print(df.info())
print("\n前几行数据:")
print(df.head())

# # 检查每列的空值数量
# print("每列的空值数量:")
# print(df.isnull().sum())

# # 检查总的空值数量
# print("\n总空值数量:", df.isnull().sum().sum())

# # 检查哪些列的空值比例
# print("\n每列空值比例:")
# print(df.isnull().sum() / len(df) * 100)

# # 找出包含空值的行
# null_rows = df[df.isnull().any(axis=1)]
# print("\n包含空值的行数:", len(null_rows))

DataFrame 信息:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45962 entries, ('110038.SH', datetime.date(2022, 8, 1)) to ('111007.SH', datetime.date(2022, 12, 30))
Data columns (total 78 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   name                  45962 non-null  object        
 1   pre_close             45962 non-null  float64       
 2   open                  45962 non-null  float64       
 3   high                  45962 non-null  float64       
 4   low                   45962 non-null  float64       
 5   close                 45962 non-null  float64       
 6   limit_up_down         45962 non-null  int64         
 7   close_ma_5            45962 non-null  float64       
 8   bias_5                45962 non-null  float64       
 9   pct_chg               45962 non-null  float64       
 10  adj_factor            45962 non-null  float64       
 11  vol                   45962 non-null  floa

In [11]:
df.columns

Index(['name', 'pre_close', 'open', 'high', 'low', 'close', 'limit',
       'close_ma_5', 'bias_5', 'pct_chg', 'adj_factor', 'vol', 'vol_5',
       'amount', 'amount_5', 'volatility', 'code_stk', 'name_stk',
       'pre_close_stk', 'open_stk', 'high_stk', 'low_stk', 'close_stk',
       'pct_chg_stk', 'adj_factor_stk', 'vol_stk', 'amount_stk', 'pe_ttm',
       'pb', 'ps_ttm', 'dv_ratio', 'total_share', 'float_share', 'total_mv',
       'circ_mv', 'debt_to_assets', 'volatility_stk', 'is_call', 'conv_price',
       'conv_value', 'conv_prem', 'dblow', 'issue_size', 'remain_size',
       'remain_cap', 'turnover', 'turnover_5', 'cap_mv_rate', 'list_date',
       'list_days', 'conv_start_date', 'left_conv_start_days', 'conv_end_date',
       'left_years', 'ytm', 'pure_value', 'bond_prem', 'option_value',
       'theory_value', 'theory_bias', 'rating', 'yy_rating', 'orgform', 'area',
       'industry_1', 'industry_2', 'industry_3', 'maturity_put_price',
       'maturity', 'popularity_ranking',

In [ ]:
create_schema_sql = "CREATE SCHEMA IF NOT EXISTS cb_schema;"
    
create_table_sql = """
    CREATE TABLE IF NOT EXISTS cb_schema.cb_data (
        id BIGINT AUTO_INCREMENT PRIMARY KEY,
        trade_date DATE NOT NULL,
        code VARCHAR(10) NOT NULL,
        name VARCHAR(50),
        pre_close DECIMAL(10,4),
        open DECIMAL(10,4),
        high DECIMAL(10,4),
        low DECIMAL(10,4),
        close DECIMAL(10,4),
        limit_up_down TINYINT,  -- 改名为 limit_flag
        close_ma_5 DECIMAL(10,4),
        bias_5 DECIMAL(10,4),
        pct_chg DECIMAL(10,4),
        adj_factor DECIMAL(10,4),
        vol DECIMAL(20,4),
        vol_5 DECIMAL(20,4),
        amount DECIMAL(20,4),
        amount_5 DECIMAL(20,4),
        volatility DECIMAL(10,4),
        code_stk VARCHAR(10),
        name_stk VARCHAR(50),
        pre_close_stk DECIMAL(10,4),
        open_stk DECIMAL(10,4),
        high_stk DECIMAL(10,4),
        low_stk DECIMAL(10,4),
        close_stk DECIMAL(10,4),
        pct_chg_stk DECIMAL(10,4),
        adj_factor_stk DECIMAL(10,4),
        vol_stk DECIMAL(20,4),
        amount_stk DECIMAL(20,4),
        pe_ttm DECIMAL(10,4),
        pb DECIMAL(10,4),
        ps_ttm DECIMAL(10,4),
        dv_ratio DECIMAL(10,4),
        total_share DECIMAL(20,4),
        float_share DECIMAL(20,4),
        total_mv DECIMAL(20,4),
        circ_mv DECIMAL(20,4),
        debt_to_assets DECIMAL(10,4),
        volatility_stk DECIMAL(10,4),
        is_call VARCHAR(20),
        conv_price DECIMAL(10,4),
        conv_value DECIMAL(10,4),
        conv_prem DECIMAL(10,4),
        dblow DECIMAL(10,4),
        issue_size DECIMAL(20,4),
        remain_size DECIMAL(20,4),
        remain_cap DECIMAL(20,4),
        turnover DECIMAL(10,4),
        turnover_5 DECIMAL(10,4),
        cap_mv_rate DECIMAL(10,4),
        list_date DATE,
        list_days INT,
        conv_start_date DATE,
        left_conv_start_days INT,
        conv_end_date DATE,
        left_years DECIMAL(10,4),
        ytm DECIMAL(10,4),
        pure_value DECIMAL(10,4),
        bond_prem DECIMAL(10,4),
        option_value DECIMAL(10,4),
        theory_value DECIMAL(10,4),
        theory_bias DECIMAL(10,4),
        rating VARCHAR(10),
        yy_rating VARCHAR(10),
        orgform VARCHAR(20),
        area VARCHAR(20),
        industry_1 VARCHAR(50),
        industry_2 VARCHAR(50),
        industry_3 VARCHAR(50),
        maturity_put_price DECIMAL(10,4),
        maturity DECIMAL(10,4),
        popularity_ranking INT,
        pct_chg_5 DECIMAL(10,4),
        pct_chg_5_stk DECIMAL(10,4),
        alpha_pct_chg_5 DECIMAL(10,4),
        theory_conv_prem DECIMAL(10,4),
        mod_conv_prem DECIMAL(10,4),
        open_pct_chg DECIMAL(10,4),
        high_pct_chg DECIMAL(10,4),
        low_pct_chg DECIMAL(10,4),
        
        INDEX idx_trade_date (trade_date),
        INDEX idx_code (code),
        INDEX idx_code_stk (code_stk),
        UNIQUE INDEX idx_code_date (code, trade_date)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """



// 上传转债数据
upload_cb_data_to_db('20220101', '20230101')

// 更新转债数据
# 从您之前的输出中，我们可以看到一些列确实存在空值：
# 1. pe_ttm: 391个非空值，意味着有 507-391=116 个空值
# 2. pb: 506个非空值，意味着有 507-506=1 个空值
# 3. dv_ratio: 461个非空值，意味着有 507-461=46 个空值
# 4. is_call: 87个非空值，意味着有 507-87=420 个空值


# SELECT code, trade_date, name, close, code_stk, name_stk, close_stk, pe_ttm, industry_1, industry_2, dv_ratio, is_call
# FROM cb_schema.cb_data 
# WHERE trade_date BETWEEN '20180101' AND '20260102'
# AND (is_call IS NULL OR is_call = '')
# ORDER BY trade_date, code;

# UPDATE cb_schema.cb_data
# SET pe_ttm = -1
# WHERE trade_date BETWEEN '20180101' AND '20260102'
#   AND pe_ttm IS NULL;

# UPDATE cb_schema.cb_data
# SET pb = -1
# WHERE trade_date BETWEEN '20180101' AND '20260102'
#   AND pb IS NULL;

# dv_ratio = 0 is ok.

# is_call:
# (NULL)	
# 433334

# 公告不强赎

# 55835

# 公告到期赎回

# 1247

# 公告实施强赎

# 4772

# 公告提示强赎

# 1018

# 已满足强赎条件

# 4707

# 未到转股期

# 91842


